In [1]:
from dotenv import load_dotenv

assert load_dotenv("../../.env")

In [2]:
import ragna

ragna.__version__

'0.1.dev57+g0e9c075.d20231025200357'

In [3]:
from ragna import Config

config = Config()
config

Config(local_cache_root=PosixPath('/home/philip/.cache/ragna'), core=CoreConfig(queue_url='memory', document=<class 'ragna.core.LocalDocument'>, source_storages=[<class 'ragna.source_storages.RagnaDemoSourceStorage'>], assistants=[<class 'ragna.assistants.RagnaDemoAssistant'>]), api=ApiConfig(url='http://127.0.0.1:31476', database_url='memory', authentication=<class 'ragna.core.RagnaDemoAuthentication'>, upload_token_secret='thiZLJBgUNrJyJ150sfWrEFSkn8SR-jH', upload_token_ttl=300), ui=UiConfig(url='http://127.0.0.1:31477'))

In [4]:
document_path = "demo_document.txt"

with open(document_path, "w") as file:
    file.write("Ragna is an open-source RAG orchestration app\n")

In [5]:
from ragna.assistants import RagnaDemoAssistant
from ragna.core import Rag
from ragna.source_storages import RagnaDemoSourceStorage

rag = Rag(config)

async with rag.chat(
    documents=[document_path],
    source_storage=RagnaDemoSourceStorage,
    assistant=RagnaDemoAssistant,
) as chat:
    prompt = "What is Ragna?"
    answer = await chat.answer(prompt)

print(answer)

I can't really help you with your prompt:

> What is Ragna?

I can at least show you the sources that I was given:

- demo_document.txt: Ragna is an open-source RAG orchestration app


In [6]:
import asyncio
import itertools
from pprint import pprint

from ragna.assistants import (
    Claude,
    ClaudeInstant,
    Gpt4,
    Gpt35Turbo16k,
    Mpt7bInstruct,
    Mpt30bInstruct,
)
from ragna.source_storages import Chroma, LanceDB

source_storages = [Chroma, LanceDB]
assistants = [
    Gpt35Turbo16k,
    Gpt4,
    ClaudeInstant,
    Claude,
    Mpt7bInstruct,
    Mpt30bInstruct,
]


async def answer_prompt(source_storage, assistant):
    async with rag.chat(
        documents=[document_path],
        source_storage=source_storage,
        assistant=assistant,
    ) as chat:
        message = await chat.answer(prompt)
        return message.content


experiments = {
    (source_storage.display_name(), assistant.display_name()): answer_prompt(
        source_storage, assistant
    )
    for source_storage, assistant in itertools.product(source_storages, assistants)
}

pprint(experiments)

{('Chroma', 'Anthropic/claude-2'): <coroutine object answer_prompt at 0x7f850cb4b6c0>,
 ('Chroma', 'Anthropic/claude-instant-1'): <coroutine object answer_prompt at 0x7f850cb4b640>,
 ('Chroma', 'MosaicML/mpt-30b-instruct'): <coroutine object answer_prompt at 0x7f850cb4b7c0>,
 ('Chroma', 'MosaicML/mpt-7b-instruct'): <coroutine object answer_prompt at 0x7f850cb4b740>,
 ('Chroma', 'OpenAI/gpt-3.5-turbo-16k'): <coroutine object answer_prompt at 0x7f850cb4b0c0>,
 ('Chroma', 'OpenAI/gpt-4'): <coroutine object answer_prompt at 0x7f850cb4b5c0>,
 ('LanceDB', 'Anthropic/claude-2'): <coroutine object answer_prompt at 0x7f850cb4b9c0>,
 ('LanceDB', 'Anthropic/claude-instant-1'): <coroutine object answer_prompt at 0x7f850cb4b940>,
 ('LanceDB', 'MosaicML/mpt-30b-instruct'): <coroutine object answer_prompt at 0x7f850cb4bac0>,
 ('LanceDB', 'MosaicML/mpt-7b-instruct'): <coroutine object answer_prompt at 0x7f850cb4ba40>,
 ('LanceDB', 'OpenAI/gpt-3.5-turbo-16k'): <coroutine object answer_prompt at 0x7f850

In [7]:
results = dict(zip(experiments.keys(), await asyncio.gather(*experiments.values())))
pprint(results)

{('Chroma', 'Anthropic/claude-2'): ' Based on the context provided, Ragna is '
                                   'an open-source RAG orchestration app.',
 ('Chroma', 'Anthropic/claude-instant-1'): ' Based on the context provided, '
                                           'Ragna is an open-source RAG '
                                           'orchestration app.',
 ('Chroma', 'MosaicML/mpt-30b-instruct'): 'Ragna is an open-source RAG '
                                          'orchestration app',
 ('Chroma', 'MosaicML/mpt-7b-instruct'): 'Ragna is an open-source RAG '
                                         'orchestration app',
 ('Chroma', 'OpenAI/gpt-3.5-turbo-16k'): 'Ragna is an open-source RAG '
                                         '(Retrieve, Ask, Generate) '
                                         'orchestration app. It is designed to '
                                         'facilitate the creation and '
                                         'deployment of convers